In [292]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Scrapping and creating Dataframe

In [293]:
results = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(results, 'lxml')
table = soup.find('table', class_='wikitable').tbody
postCode=[]
borough = []
neighborhood = []
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
for tr in table.find_all('tr'):
    td = tr.find_all('td')
    if td:
        postCode.append(td[0].text)
        borough.append(td[1].text)
        neighborhood.append(td[2].text[:-1])

In [294]:
df['PostalCode'], df['Borough'], df['Neighborhood'] = postCode, brough, neighborhood

In [295]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


# Filtering data frame for Not assigned Borough and Neighborhood

In [340]:
df = df[df.Borough !='Not assigned']
df.Neighborhood = df.Borough.where(df.Neighborhood =='Not assigned', df.Neighborhood)
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [341]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()

In [342]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [343]:
df.shape

(103, 3)

## Fetch latitude and longitude using geocoder

In [344]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='foursquare')
location = geolocator.geocode("M1B, Rouge,Malvern")
location

Location(Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada, (43.8091955, -79.2217008, 0.0))

In [345]:
# def getGeoCodes(row):
#     loc = None
#     while not loc:
#         loc = geolocator.geocode('{}, {}'.format(row['PostalCode'], row['Neighborhood']))
#         print(row['PostalCode'], row['Neighborhood'],loc)
#     return loc.latitude, loc.longitude

# df[['PostalCode', 'Neighborhood']].apply(getGeoCodes, axis=1)

In [346]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [347]:
geo_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df = df.join(geo_df.set_index('PostalCode'), on='PostalCode')

In [351]:
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
